First, we need to add the current folder to the PYTHONPATH in the code with the absolute location for the torchMPS library! wd

In [1]:
!pip install "git+https://github.com/jemisjoky/TorchMPS.git"

import torch
from torch_intermediate_layer_getter import IntermediateLayerGetter as MidGetter
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torchmps import MPS

  Running command git clone --filter=blob:none --quiet https://github.com/jemisjoky/TorchMPS.git 'C:\Users\piche\AppData\Local\Temp\pip-req-build-y5bj_tq7'


Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/jemisjoky/TorchMPS.git to c:\users\piche\appdata\local\temp\pip-req-build-y5bj_tq7
  Resolved https://github.com/jemisjoky/TorchMPS.git to commit 6c0bc1a8e2c15acba8570ca9ffe2b4a0c7135165
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Created wheel for torchmps: filename=torchmps-0.1.0-py3-none-any.whl size=60029 sha256=ff2885ab9f71ebc9cd1f9e3937b2f4656af1aea0c612d4ab5d7cb9c78998f11e
  Stored in directory: C:\Users\piche\AppData\Local\Temp\pip-ephem-wheel-cache-hqdntcp9\wheels\18\0e\5f\7e55dd9989ef77b997aac9759dd6a1ccf0a8a5b8cecef5dce9
Successfully built torchmps


c:\ProgramData\miniconda3\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Now, let us first build a fully connected simple layer by layer approach to the procedure. 

We shall use the classic MNIST dataset in order to train our the FCNN, which 

In [11]:
# Pick the hyperparameters for the FCNN
chosen_device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
n_epochs_fcnn = 1
hidd_ls = 50 #size of the hidden layers
chosen_bs = 150
chosen_loss = nn.CrossEntropyLoss()
#Optimizer parameters
chosen_lr = 0.001
chosen_momentum = 0.9

In [12]:
# Import the mnist dataset
train_set = torchvision.datasets.MNIST(root = './datasets', train = True, transform = transforms.ToTensor(),  download = True )
train_iterator = torch.utils.data.DataLoader(dataset = train_set, batch_size=chosen_bs, shuffle=True)

test_set = torchvision.datasets.MNIST(root = './datasets', train = False, transform = transforms.ToTensor(),  download = True)
test_iterator = torch.utils.data.DataLoader(dataset = test_set, batch_size= chosen_bs)


In [ ]:
# Create and train our FCNN
class FCNN(nn.Module):
    def __init__(self):
        super(FCNN, self).__init__()
        self.lin1 = nn.Linear(784, hidd_ls)
        self.relu1 = nn.ReLU()

        self.lin2 = nn.Linear(hidd_ls, hidd_ls)
        self.relu2 = nn.ReLU()

        self.lin3 = nn.Linear(hidd_ls, hidd_ls)
        self.relu3 = nn.ReLU()

        self.lin4 = nn.Linear(hidd_ls, 10)

    def forward(self, x):
        y = self.lin1(x)
        y = self.relu1(y)

        y = self.lin2(y)
        y = self.relu2(y)

        y = self.lin3(y)
        y = self.relu3(y)
        
        return y

#Instantiate and put the model on the chosen device
fcnn_teacher = FCNN().to(chosen_device)

#Instantiate the optimizer
fcnn_optimizer = torch.optim.SGD(fcnn_teacher.parameters(), lr=chosen_lr, momentum=chosen_momentum)

#Training loop
for epoch in range(n_epochs_fcnn):
    for i, (x_mb, y_mb) in enumerate(train_iterator):

        # Reshape the train_tuple and put on the chosen device
        x_mb = x_mb.reshape(-1, 784).to(chosen_device)
        y_mb = y_mb.to(chosen_device)

        # Foward propagation
        y_hat_mb = fcnn_teacher(x_mb)
        loss = chosen_loss(y_hat_mb, y_mb)

        # Backpropagation
        loss.backward()
        fcnn_optimizer.step()
        fcnn_optimizer.zero_grad()

        if (i % 1000 == 0):
            print(loss.item())


In [ ]:
# Compute the accuracy of our FCNN
with torch.no_grad():
    acc_count = 0
    for (x_mb, y_mb) in test_iterator:
        x_mb = x_mb.reshape(-1, 784).to(chosen_device)
        y_mb = y_mb.to(chosen_device)

        y_mb_hat = fcnn_teacher(x_mb) #predict

        (wedontcare, y_mb_hat_class) = torch.max(y_mb_hat, 1)
        acc_count += (y_mb_hat_class == y_mb).sum().item()

    print("Accuracy: " + str(100* acc_count / len(test_iterator.dataset)))

Now, we shall create the MPS

In [19]:
# MPS parameters
bond_dim = 20
adaptive_mode = False
periodic_bc = False

lmps = MPS(
    input_dim=28 ** 2,
    output_dim=10,
    bond_dim = bond_dim1,
    adaptive_mode=adaptive_mode,
    periodic_bc=periodic_bc,
)

#Instantiate the optimizer
lmps_optimizer = torch.optim.SGD(lmps.parameters(), lr=chosen_lr, momentum=chosen_momentum)

#Training loop
for epoch in range(n_epochs_fcnn):
    for i, (x_mb, y_mb) in enumerate(train_iterator):

        # Reshape the train_tuple and put on the chosen device
        x_mb = x_mb.reshape(-1, 784).to(chosen_device)
        y_mb = y_mb.to(chosen_device)

        # Foward propagation
        y_hat_mb = mps(x_mb)
        loss = chosen_loss(y_hat_mb, y_mb)

        # Backpropagation
        loss.backward()
        lmps_optimizer.step()
        lmps_optimizer.zero_grad()

        if (i % 1000 == 0):
            print(loss.item())

2.3025853633880615
